In [2]:
import numpy as np
import pandas as pd

In [3]:
basic_names = ["user_id","version","minutes_play","day_1_active","day_7_active"]


In [4]:
it = 0
def Generate_Sample_data(number_of_datapoints, version):
    global it
    data = np.array([1000*it + np.arange(num_of_datapoints)+1,
                     [version]*num_of_datapoints,
                     np.random.random(size =(num_of_datapoints)) * 10 ,
                     np.random.randint(2,size = num_of_datapoints),
                     np.random.randint(2,size = num_of_datapoints)]).T
    it+=1
    return data
def create_dataframe(num_of_datapoints,version,feature_names):
    return pd.DataFrame(
    data = Generate_Sample_data(num_of_datapoints,version = version),
    columns= feature_names)
    
    
def generate_dataset(num_of_groups,num_of_datapoints,feature_names):
    control_group = create_dataframe(num_of_datapoints,"control",feature_names)
    Treatment_groups = [create_dataframe(num_of_datapoints,
                                     "treatment_"+str(i+1),feature_names) for i in range(num_of_groups)]
    return pd.concat([control_group,pd.concat(Treatment_groups)])
def generte_treatment_results(num_of_groups,num_of_datapoints,feature_names):
    return np.array([np.random.randint(2,size = num_of_datapoints) for i in range(num_of_groups)])

In [5]:
num_of_groups = int(input("Input number of groups"))
feature_names = basic_names
num_of_datapoints =  int(input("Input number of datapoint per group"))
data = generate_dataset(num_of_groups,num_of_datapoints,feature_names) 
results = generte_treatment_results(num_of_groups,num_of_datapoints,feature_names) 

Input number of groups5
Input number of datapoint per group1000


In [6]:
data.head(),data.tail()

(  user_id  version        minutes_play day_1_active day_7_active
 0       1  control  1.8845875816666124            0            0
 1       2  control  2.8819411874282497            0            0
 2       3  control   6.784765599708051            1            0
 3       4  control   4.677627935507863            1            0
 4       5  control   2.541806612765003            1            1,
     user_id      version         minutes_play day_1_active day_7_active
 995    5996  treatment_5  0.36713858983651093            1            0
 996    5997  treatment_5    8.344191460287394            0            1
 997    5998  treatment_5    9.742462289254096            1            1
 998    5999  treatment_5    5.901575378740464            0            0
 999    6000  treatment_5   2.7253931974605217            0            0)

In [7]:
results[:5]

array([[0, 0, 1, ..., 1, 0, 0],
       [1, 1, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 1, 1],
       [0, 1, 1, ..., 1, 0, 0]])

In [12]:
import pandas as pd
import scipy.stats as stats

In [15]:
fvalue, pvalue = stats.f_oneway(data['day_1_active'], data['day_7_active'], data['minutes_play'])
print(fvalue, pvalue)


13453.495482857104 0.0


In [39]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.regression.linear_model import OLS

In [ ]:
model = data.ols(formula='day_1_active ~ version', data=data).fit()
aov_table = anova_lm(model, typ=2)
print(np.round(aov_table,4))